#### Importiamo le librerie necessarie. Useremo ResNet50.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import os
from tensorflow import keras
from tensorflow.keras.preprocessing import image as kimage
from tensorflow.keras.applications import resnet50, inception_v3
import pandas as pd
import datetime, os
import shutil
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

# Load the TensorBoard notebook extension
%load_ext tensorboard

keras.__version__

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile, os
# this would take very long !!!
base_dir = '/content/drive/MyDrive/DSIM_project/images_animals10_split.zip'
zip = zipfile.ZipFile(f'{base_dir}')
zip.extractall()
zip.close()

## Creiamo train e test set

In [ ]:
dest_train = "images_animals10_split/train/"
dest_test = "images_animals10_split/test/"
dest_val = "images_animals10_split/val/"

## Prima configurazione

#### Creiamo i generators

In [ ]:
train_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function= resnet50.preprocess_input,
                                                                shear_range=0.2,
                                                                zoom_range=0.2,
                                                                horizontal_flip=True,
                                                                rotation_range=20,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.2,
                                                                brightness_range=[0.5, 1.5])


train_generator = train_processing.flow_from_directory(directory = dest_train,
                                    target_size = (224, 224),
                                    color_mode = "rgb",
                                    batch_size = 32,
                                    class_mode = "categorical",
                                    shuffle = True,
                                    seed = 1234)

In [ ]:
val_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=resnet50.preprocess_input)

val_generator = val_processing.flow_from_directory(
        directory=dest_val,
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True
)

In [ ]:
test_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=resnet50.preprocess_input)

test_generator = test_processing.flow_from_directory(
        directory=dest_test,
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True
)

#### Importiamo il modello con i pesi adatti (imagenet), e settiamo i layer iniziali come non allenabili

In [ ]:
base_net = resnet50.ResNet50(weights="imagenet", include_top=False,
	input_shape=(224, 224, 3), pooling="avg")

In [ ]:
for layer in base_net.layers:
    layer.trainable = False

#### Ora aggiungiamo due fully connected layers e poi creiamo un nuovo modello "net", dato dalla combinazione dei due 

In [ ]:
# Output of the base_net model
x = base_net.output
# intermediate fully-connected layer + ReLU
x = keras.layers.Dense(1024, activation='relu')(x)
# final fully-connected layer + SoftMax 
pred = keras.layers.Dense(n_species, activation='softmax')(x)

In [ ]:
net1 = keras.Model(inputs=base_net.input, outputs=pred)

In [ ]:
net1.summary()

#### Compiliamo il modello usando come loss la categorical crossentropy, come optimizer RMSprop e come metrica l'accuracy

In [ ]:
net1.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=keras.optimizers.RMSprop(),
            metrics=['accuracy'])

#### Ora fittiamo il modello con 10 epoche e visualizziamo i risultati in termini di accuracy e loss su train e test set

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="bidim_weights",
                                                 save_weights_only=True,
                                                 monitor="val_acc",
                                                 verbose=1)


In [ ]:
history1 = net1.fit(train_generator,
          epochs=10,
          validation_data=val_generator,
          callbacks=[tensorboard_callback, cp_callback])

In [ ]:
#Open tensorboard
#Open localhost:6006

In [ ]:
def accuracy_loss(history):
    plt.figure(figsize=(20,10))
    #
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label = "train_accuracy", color = "red")
    plt.plot(history.history['val_accuracy'], label="train_accuracy", color ="blue")
    plt.legend(loc='best', fontsize = 12)
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.xlabel('Epochs', size = 15)
    plt.ylabel('Accuracy', size = 15)
    #
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label = "Train loss", color = "red")
    plt.plot(history.history['val_loss'], label="Test loss", color = "blue")
    plt.legend(loc='best', fontsize = 12)
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.xlabel('Epochs', size = 15)
    plt.ylabel('Loss', size = 15)

In [ ]:
json_config = net1.to_json("bidim_model")

load_status = net1.load_weights("bidim_weights")
load_status.assert_consumed()

In [ ]:
performance1 = net1.evaluate(test_generator)

In [ ]:
accuracy_loss(history1)